<a href="https://colab.research.google.com/github/tanmay1240/LLM/blob/main/Brain_Eyes_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multimodal Emotion Detection System – Version 2
### Emotion recognition using Vision + Caption + VQA + FER fusion


In [ ]:
pip install transformers torch


In [ ]:
from transformers import pipeline
from PIL import Image
import re
from transformers import CLIPProcessor, CLIPModel
import torch

# =====================================================
# 1. Load Vision Model (BLIP)
# =====================================================
captioner = pipeline(
    "image-to-text",
    model="Salesforce/blip-image-captioning-large"
)

# =====================================================
# 1.5 Load VQA Model (Emotion Probe)
# =====================================================
vqa = pipeline(
    "visual-question-answering",
    model="Salesforce/blip-vqa-base"
)

# =====================================================
# 1.6 Load CLIP Model
# =====================================================
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# =====================================================
# 2. Load LLM
# =====================================================
llm = pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    max_new_tokens=120,
    temperature=0.1,
    do_sample=False,
    return_full_text=False
)

# =====================================================
# 3. Load Image and Generate Caption
# =====================================================
image_path = input("Enter the image path of the image you want to analyze: ")

try:
    image = Image.open(image_path)
except:
    print("❌ Error: Could not open the image. Please check the path.")
    exit()

caption_result = captioner(image)
caption = caption_result[0]["generated_text"]

print("\n🔍 Image Caption:")
print(caption)
print("-" * 50)

# -------------------------------
# VQA Emotion Hint (Moved here after image is loaded)
# -------------------------------
vqa_result = vqa(
    image=image,
    question="What emotion is the person showing?"
)

vqa_emotion = vqa_result[0]["answer"].lower()

print("\n🎯 VQA Emotion Hint:")
print(vqa_emotion)

# -------------------------------
# CLIP Emotion Prediction
# -------------------------------
clip_prompts = [
    "a person who is happy",
    "a person who is sad",
    "a person who is angry",
    "a person who is afraid",
    "a person with a neutral expression"
]

inputs = clip_processor(
    text=clip_prompts,
    images=image,
    return_tensors="pt",
    padding=True
)

with torch.no_grad():
    outputs = clip_model(**inputs)
    logits_per_image = outputs.logits_per_image
    probs = logits_per_image.softmax(dim=1)

clip_index = probs.argmax().item()
clip_emotion = clip_prompts[clip_index].split("who is ")[-1].replace("a person ", "").strip()

# Normalize
clip_emotion = clip_emotion.replace("with a ", "").replace(" expression", "").capitalize()

print("\n🎯 CLIP Emotion Prediction:")
print(clip_emotion)

# =====================================================
# 4. Build Prompt for LLM
# =====================================================
prompt = f"""
You MUST return output in exactly this format:
Human: Yes/No
Emotion: one of [Happy, Sad, Angry, Neutral, Fear, Surprise, Not Applicable]
Reasoning: one sentence explanation

Rules:
- Exactly ONE output block.
- No repetition.
- No additional examples.
- Do NOT invent any story or context.
- Do NOT add information not present in the description.
- Do NOT infer emotion from pets, animals, environment, or objects.
- Only use explicit emotional cues from the description.
- Concerned, worried, nervous → map to Fear.
- If no clear emotion matches, use Neutral.
- If no human is present, Emotion = Not Applicable.

Description:
{caption}
"""

# =====================================================
# 5. Run LLM
# =====================================================
llm_result = llm(prompt)
raw_output = llm_result[0]["generated_text"]

# =====================================================
# 6. Extract First Valid Answer Block
# =====================================================
pattern = r"Human:\s.*?\nEmotion:\s.*?\nReasoning:.*?(?=\nHuman:|\Z)"
match = re.search(pattern, raw_output, re.DOTALL)

if match:
    answer = match.group(0).strip()
else:
    answer = raw_output.strip()

# =====================================================
# 7. Parse Fields
# =====================================================
human_match = re.search(r"Human:\s*(Yes|No)", answer, re.IGNORECASE)
emotion_match = re.search(r"Emotion:\s*([\w\s]+)", answer, re.IGNORECASE)
reasoning_match = re.search(r"Reasoning:\s*(.*)", answer, re.DOTALL | re.IGNORECASE)

human = human_match.group(1).capitalize() if human_match else "Unknown"
emotion = emotion_match.group(1).strip().capitalize() if emotion_match else "Unknown"
reasoning = reasoning_match.group(1).strip() if reasoning_match else ""

# =====================================================
# 8. Correct Human Detection Logic (Priority Based)
# =====================================================
caption_lower = caption.lower()

human_keywords = [
    "person", "people", "man", "woman",
    "boy", "girl", "child", "couple",
    "human", "face"
]

non_human_keywords = [
    "car", "vehicle", "building", "street", "road",
    "house", "tree", "nature", "landscape",
    "object", "sky", "cloud", "mountain"
]

# Priority:
# 1. If human keywords → Human = Yes
# 2. Else if non-human keywords → Human = No
# 3. Else → Unknown
if any(word in caption_lower for word in human_keywords):
    human = "Yes"
elif any(word in caption_lower for word in non_human_keywords):
    human = "No"
else:
    human = "Unknown"

# =====================================================
# Hard Override for Non-Human Images
# =====================================================
if human == "No":
    emotion = "Not Applicable"
    reasoning = "No humans are present in the image, so emotion classification is not applicable."

# =====================================================
# 8.5 Emotion Refinement Layer (Caption-Only + Priority)
# =====================================================
emotion_map = {
    "Angry": [
        "angry", "anger", "furious", "annoyed",
        "shouting", "yelling", "screaming",
        "scowl", "scowling", "glare", "glaring",
        "hostile", "aggressive", "rage"
    ],
    "Fear": [
        "fear", "afraid", "scared", "worried",
        "nervous", "anxious", "concerned",
        "terrified", "panic"
    ],
    "Sad": [
        "cry", "crying", "sad", "sadness",
        "upset", "teary", "tear", "tears", "weeping"
    ],
    "Happy": [
        "smile", "smiling", "laugh", "laughing",
        "cheerful", "joyful"
    ]
}

# Priority order: Angry > Fear > Sad > Happy
emotion_priority = ["Angry", "Fear", "Sad", "Happy"]

detected_emotion = None
for emotion_label in emotion_priority:
    keywords = emotion_map[emotion_label]
    if any(word in caption_lower for word in keywords):
        detected_emotion = emotion_label
        break

# =====================================================
# Emotion Fusion Logic (Caption → VQA → CLIP)
# =====================================================

valid_emotions = ["Happy", "Sad", "Angry", "Fear", "Neutral"]

if human == "Yes":
    # 1. Caption-based emotion (highest priority)
    if detected_emotion:
        emotion = detected_emotion
        reasoning = (
            f"The description contains explicit emotional cues related to {emotion.lower()}, "
            f"so the emotion is classified as {emotion}."
        )

    # 2. VQA-based emotion
    elif vqa_emotion in [e.lower() for e in valid_emotions]:
        emotion = vqa_emotion.capitalize()
        if emotion == "Scared":
            emotion = "Fear"

        reasoning = (
            f"The visual question answering model suggests the emotion '{emotion}', "
            f"which is used when the caption does not provide explicit emotional cues."
        )

    # 3. CLIP-based emotion
    elif clip_emotion in valid_emotions:
        emotion = clip_emotion
        reasoning = (
            f"The CLIP model shows highest similarity between the image and the concept '{emotion}', "
            f"so the emotion is classified as {emotion}."
        )

    # 4. Fallback
    else:
        emotion = "Neutral"
        reasoning = (
            "Neither the caption, the visual question answering model, nor the CLIP model "
            "provides strong emotional evidence, so the emotion is classified as Neutral."
        )
else:
    emotion = "Not Applicable"
    reasoning = "No humans are present in the image, so emotion classification is not applicable."


# =====================================================
# 9. Prevent Story Hallucination
# =====================================================
forbidden_words = ["home", "work", "school", "office", "after", "day"]

if any(word in reasoning.lower() for word in forbidden_words):
    reasoning = (
        "The description contains a human, but no contextual background or life situation "
        "is provided. The reasoning must rely only on the visual description."
    )

# =====================================================
# 10. Final Output
# =====================================================
final_output = f"""
🧠 Multimodal Reasoning Output
-----------------------------
Human: {human}
Emotion: {emotion}
Reasoning: {reasoning}
"""

print(final_output)
